In [1]:
import parselmouth
from parselmouth.praat import call
import numpy as np
import tensorflow as tf
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('/content/drive/MyDrive/PDData.csv')

In [3]:
df = df.drop('Unnamed: 0', axis = 1)

In [4]:
X = df.drop('status',axis=1)
y = df['status']

In [5]:
scale = StandardScaler()
X = scale.fit_transform(X)
y = y


In [6]:
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [7]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(0.01)))

In [8]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(X_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
20/20 [==============================] - 2s 3ms/step - loss: 0.7289 - accuracy: 0.2628
Epoch 2/100
20/20 [==============================] - 0s 6ms/step - loss: 0.7078 - accuracy: 0.3141
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6942 - accuracy: 0.4744
Epoch 4/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6848 - accuracy: 0.6795
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6768 - accuracy: 0.7949
Epoch 6/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6695 - accuracy: 0.8141
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6642 - accuracy: 0.8269
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6591 - accuracy: 0.8462
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6542 - accuracy: 0.8526
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6502 - accuracy: 0.8397
Epoch 11/

In [10]:
loss, accuracy = model.evaluate(X_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

2/2 [==============================] - 0s 9ms/step - loss: 0.4700 - accuracy: 0.8974
Loss: 0.4699670970439911
Accuracy: 0.8974359035491943


In [11]:
def measurePitch(voiceID, f0min, f0max, unit):
    sound = parselmouth.Sound(voiceID) # read the sound
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object
    meanF0 = call(pitch, "Get mean", 0, 0, unit) # get mean pitch
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    hnr = call(harmonicity, "Get mean", 0, 0)
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
    ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)
    localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    

    return meanF0, rapJitter , localabsoluteJitter, ppq5Jitter, ddpJitter, localdbShimmer, apq3Shimmer, aqpq5Shimmer, ddaShimmer, hnr

In [12]:
wave_file = r'/content/drive/MyDrive/New_Recording_8.wav'
sound = parselmouth.Sound(wave_file)
(meanF0, rapJitter, localabsoluteJitter, ppq5Jitter, ddpJitter, localdbShimmer, apq3Shimmer, aqpq5Shimmer, ddaShimmer, hnr) = measurePitch(sound, 75, 500, "Hertz")


In [13]:
test_sound_m = [meanF0, rapJitter, localabsoluteJitter, ppq5Jitter, ddpJitter, localdbShimmer, apq3Shimmer, aqpq5Shimmer, ddaShimmer, hnr]

In [14]:
model.predict(np.array(test_sound_m).reshape(1,-1))

1/1 [==============================] - 0s 149ms/step


array([[0.42670012]], dtype=float32)

In [15]:
model.save('VocalModel.h5')